# Words selection from handwriting documents

Import libraries

In [63]:
import os
import sys
import cv2
import random
import math
import itertools
import numpy as np
import matplotlib.pylab as plt
from matplotlib.pyplot import plot
import statistics   
from statistics import mean
from collections import namedtuple

Functions to show image

In [64]:
GREEN = (0, 255, 0)
BLUR = (255, 0, 0)
RED = (0, 0, 255)

def show(img):
    """show rgb image"""
    ax = plt.axes([0,0,4,4], frameon=False)
    ax.set_axis_off()
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show()
    
def show_gray(img):
    """show grayscale image"""
    ax = plt.axes([0,0,4,4], frameon=False)
    ax.set_axis_off()
    plt.imshow(img, cmap='gray', vmin=0, vmax=255)
    plt.show()

In [65]:
def save_words(path, words):
    for i, word in enumerate(words):
        cv2.imwrite(os.path.join(path , 'word_' + str(i) + '.png'), word)
        
def sort_w(word):
    shape = word.shape
    return shape[1]

#### Functions to delete internal contours

In [66]:
class RECT:
    """Class that helps to  find where one contours are inside others"""
    def __init__(self, x, y, h,  w):
        self.x = x
        self.y = y
        self.h = h
        self.w = w

def is_rectangle_internal(R1,  R2):
    """If one rectangle contains another"""
    if ((R2.x+R2.w) < (R1.x+R1.w)) and ((R2.x) > (R1.x)) and ((R2.y) > (R1.y)) and ((R2.y+R2.h) < (R1.y+R1.h)):
            return True;
    else:
        return False;

def get_rectangles_from_contours(contours, h_img, w_img):
    """Get all rectangles from contours, delete internal contours"""
    rectangles = []
    for i, ctr in enumerate(contours):
        x, y, w, h = cv2.boundingRect(ctr)
        if w >= w_img/50 and h >= h_img/50 and w <= w_img/2 and h <= h_img/10:
            r = RECT(x,y,h, w)
            rectangles += [r]
    #Get only internal rectangles
    resulted_rectangles = []
    for i in range (len(rectangles)):
        isSmall = True
        r1 = rectangles[i]
        for j in range(i+1, len(rectangles)):
            r2 = rectangles[j]
            if (is_rectangle_internal(r1, r2)):
                isSmall = False       
        if isSmall:
            resulted_rectangles += [r1]
    return resulted_rectangles

#### Functions that helps to improve binarization index

In [67]:
def percent_of_white_pixels_word(thresh_image):
    """Count percent of white pixels in the concrete word"""
    white_pixels = 0
    h, w, = thresh_image.shape
    for i in range (h):
        for j in range (w):
            if (thresh_image[i][j] == 255):
                white_pixels += 1

    all_pixels = h *  w
    return (round(white_pixels/all_pixels, 2))

def percent_of_white_pixels(img, thresh_index):
    """Count percent of white pixels in the whole image"""
    print("thresh_index" + str(thresh_index))
    h, w, _ = img.shape
    # delete boundaries
    image = img[40:h-40, 40:w-40]
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    ret,thresh = cv2.threshold(gray,thresh_index,255,cv2.THRESH_BINARY_INV)
    show(thresh)
    white_pixels = 0
    h, w = thresh.shape
    for i in range (h):
        for j in range (w):
            if (thresh[i][j] == 255):
                white_pixels += 1

    all_pixels = h *  w
    return (round(white_pixels/all_pixels, 2))

        
def compare_thresh_indexes(img):
    """Determines which binarization index is the best is the best """
    #values to check which index is optimal for binarization
    thresh_indexes = [107, 127, 147, 167, 187, 207, 227 ]
    dict_with_norm_white_percent = {}
    dict_with_big_white_percent = {}
    best_percent = 100; best_index = 0; best_val = 100
    
    # Add to dictionary all values of each index binarization
    for index in thresh_indexes:
        percent = percent_of_white_pixels(img, index)
        if percent >= 0.01 and percent <= 0.15:
            dict_with_norm_white_percent[index] = percent
            print("index " + str(index) + " - " + str(percent))
        else:
            dict_with_big_white_percent[index] = percent
            
    # Select best index of binarization
    if len(dict_with_norm_white_percent) == 0:
        for index in dict_with_big_white_percent.keys():
            if dict_with_big_white_percent[index] <= best_percent:
                best_percent = dict_with_big_white_percent[index]
                best_index = index
            
    else:
        for index in dict_with_norm_white_percent.keys():
            new_value = math.fabs(dict_with_norm_white_percent[index] - 0.07)
            if new_value <= best_val:
                best_val = new_value
                best_percent = dict_with_norm_white_percent[index]
                best_index = index
    print("best index = " + str(index) + " - best_percent = " + str(best_percent))
    return index



#### Functions that helps to select words from too big contours

In [68]:
def get_thresh_image(img, index):
    if len(img.shape) == 3:
        h, w, _  = img.shape
    else:
        h, w  = img.shape
    contour = [0, 0, w, h]
    binary = cv2.cvtColor(img.copy(), cv2.COLOR_RGB2GRAY)
    show_gray(binary)
    ret,thresh = cv2.threshold(binary,index,255,cv2.THRESH_BINARY)
    show(thresh)
    return thresh

def prepare_binary_contour(contour):
    """Aggregate function to select lines"""
    hists, average_black_height, average_white_height, lines_count = build_hists(contour)

    sm_hists = smooth_hists(hists)
    #show_hists(sm_hists)
    valleys = find_valleys(sm_hists)
    #show_valleys(contour, valleys)
    lines, avg_height = get_lines(contour, valleys) 

    #show_lines(contour, lines)
    if lines != []:
        lines = filter_chunks(lines, average_black_height, average_white_height)
    else: 
        return []

    #show_lines(contour, lines)
    created_lines = get_first_approach_lines(lines, average_black_height)
    #draw_lines(contour, created_lines)

    created_lines = sorted(created_lines, key = lambda x: x.data[0].y)
    #draw_lines(contour, created_lines)
    extracted_lines = cut_lines(contour, created_lines)
    return extracted_lines


#### First approximation of lines - building histograms of image

In [69]:
def build_hists(image):
    """Build hist to each chunk"""
    hists = []
    chunks = get_chunks(image)
    black_height = []; white_height = []
    lines_count = []
    for i, val in enumerate(chunks[:-1]):
        chunk = image[:, val : chunks[i + 1]]
        hist = np.sum(1 - (chunk / 255), axis=1)
        current_black_height = 0
        current_white_height = 0
        count = 0
        if hist[0] == 0:
            current_black_height += 1
        else:
            current_white_height += 1
        for pix in hist[1:]:
            if pix == 0:
                if current_black_height > 0:
                    current_black_height += 1
                else:
                    current_black_height += 1
                    white_height.append(current_white_height)
                    current_white_height = 0
            else:
                if current_white_height > 0:
                    current_white_height += 1
                else:
                    current_white_height += 1
                    black_height.append(current_black_height)
                    count += 1
                    current_black_height = 0
        lines_count.append(count)
        hists.append(hist)
    return hists, mean(black_height), mean(white_height), lines_count

def smooth_hists(hists):
    """Smooth hists witch is got in function build_hists"""
    new_hists = []
    for hist in hists:
        new_hists.append(median_smooth(hist))
    return new_hists

def median_smooth(signal, kernel_size = 5):
    """Smooth signal"""
    smooth_signal = []
    for i, val in enumerate(signal[:-kernel_size]):
        smooth_signal.append(sum(signal[i: i + kernel_size]) / kernel_size)
    return np.array(smooth_signal)

def show_hists(hists):
    """
    Draw hists
    """
    fig = plt.figure()
    y = np.arange(len(hists[0]))
    for i in range(len(hists)):
        h = hists[i]
        plt.plot(h[::-1] + i*35, y)
    ax = plt.axes([0,0,1,1], frameon=False)
    ax.set_axis_off()
    plt.show()
    fig.savefig('temp.png', dpi=fig.dpi)
    
def find_valleys(sm_hists, thresh = 1):
    """Finding troughs on smoothed histograms"""
    valleys = []
    interval_average_height = []
    for curr in sm_hists:
        curr[curr < thresh] = 0
        valleys_ind, curr_valley = [], []
        prev = 1
        for i, val in enumerate(curr[:-1]):
            if (val == 0 and prev != 0):
                curr_valley.append(i)
            if (val == 0 and curr[i + 1] != 0):
                curr_valley.append(i)
            if len(curr_valley) == 2:
                valleys_ind.append(curr_valley)
                interval_average_height.append(curr_valley[1] - curr_valley[0])
                curr_valley = []
            if len(curr_valley) == 1 and (i == len(curr) - 2):
                curr_valley.append(i)
                valleys_ind.append(curr_valley)
            prev = val
        valleys.append(valleys_ind)
    return valleys

def show_valleys(image, valleys, channels = 2):
    chunks = get_chunks(image, channels)
    img = image.copy()
    
    for i, y in enumerate(chunks[:-1]):
        for (x_1, x_2) in valleys[i]:
            cv2.line(img, (y, x_1), (chunks[i + 1], x_1),GREEN,3)
            cv2.line(img, (y, x_2), (chunks[i + 1], x_2),RED,3)
    if channels == 3:
        show(img)
    else:
        show_gray(img)     
        
def get_lines(image, valleys, channels = 2):
    """Calculate the middle line in the trough and the average line height"""
    total_lines = []
    chunks = get_chunks(image, channels)
    for i, chunk in enumerate(chunks[:-1]):
        chunk_lines = []
        for val in valleys[i]:
            chunk_lines.append(Trait(chunk, chunks[i+1], sum(val) // len(val)))
        total_lines.append(chunk_lines)
    
    height = []
    for i, line in enumerate(total_lines):
        for j, trait in enumerate(line[:-1]):
            height.append(abs(trait.y_1 - line[j + 1].y_1))
    height = sum(height) / len(height)
    return total_lines, height

def show_lines(image, lines):
    img = image.copy()
    for chunk in lines:
        for line in chunk:
            cv2.line(img, (line.x_1, line.y_1), (line.x_2, line.y_1),GREEN,3)
    show(img)
    
def get_chunks(image, channels = 1):
    """
    Split image on chunks, each chunk is 10 percent of image width
    return: array with y-coordinates
    """
    n, m = image.shape
    return np.arange(0, m + 1, m // 10)

#### Classes that helps to improve lines selection

In [70]:
Point = namedtuple('Point', ['x' , 'y'])

class Line:
    def __init__(self):
        self.data = []
        self.last_trait = None
        
    def continue_line(self, trait):
        self.data.append(Point(trait.x_1, trait.y_1))
        self.data.append(Point(trait.x_2, trait.y_1))

class Trait:
    def __init__(self, x_1, x_2, y_1):
        self.x_1 = x_1
        self.x_2 = x_2
        self.y_1 = y_1
        #self.y_2 = y_2
    
    def dist(self, other):
        return ((self.x_2 - other.x_1) ** 2 + (self.y_1 - other.y_1) ** 2) ** (1 / 2)
    
    def print_(self):
        print('x_1: {}  x_2: {}  y: {}'.format(self.x_1, self.x_2, self.y_1))


#### Statistical functions to improve lines selection

In [71]:
def filter_chunk(chunk, avg_black_height, average_white_height):
    if chunk == []:
        return []
    prev = chunk[0]
    new_traits = [prev]
    for i, trait in enumerate(chunk[1:-1]):
        next_trait = chunk[i + 2]
        if abs(prev.y_1 - trait.y_1) > (avg_black_height + average_white_height/4) \
            and abs(next_trait.y_1 - trait.y_1) > (avg_black_height+ average_white_height/4):
            new_traits.append(trait)
            prev = trait
    new_traits.append(chunk[-1])
    return new_traits

def filter_chunks(chunks, avg_black_height, average_white_height):
    new_chunks = []
    if chunks == []:
        return []
    for chunk in chunks:
        new_chunks.append(filter_chunk(chunk, avg_black_height, average_white_height))
    return new_chunks

def get_first_approach_lines(chunk_with_traits, avg_height):
    """
    Get first approach of splitting lines
    """
    if chunk_with_traits is None:
        return []
    created_lines = connect_two_chunks(chunk_with_traits[0], chunk_with_traits[1], avg_height)
    connect_start_lines_with_next_chunks(created_lines, chunk_with_traits[2], avg_height)
    for chunk in chunk_with_traits[3:]:
        connect_lines_with_chunk(created_lines, chunk, avg_height)
    #print('avg height = ' + str(avg_height))
    return created_lines
                
def connect_two_chunks(chunk_1, chunk_2, avg_height):
    if chunk_1 == [] or chunk_2 == []:
        return []
    all_inds_from_chunk2 = [i for i in range(len(chunk_2))]
    used_traits_from_chunk2 = []
    created_lines = []
    for trait_1 in chunk_1:
        minimum = sys.maxsize
        min_trait = Trait(trait_1.x_1, trait_1.x_2, 100)
        for j, trait_2 in enumerate(chunk_2):
            if trait_1.dist(trait_2) <  minimum:
                minimum = trait_1.dist(trait_2)
                min_trait = trait_2
            if minimum >= 2 * avg_height / 3:
                min_trait = Trait(min_trait.x_1, min_trait.x_2, trait_1.y_1)
                
            if (min_trait.x_1 == trait_2.x_1) and (min_trait.x_2 == trait_2.x_2) \
                and (min_trait.y_1 == trait_2.y_1):
                used_traits_from_chunk2.append(j)
        new_line = Line()
        new_line.continue_line(trait_1)
        new_line.continue_line(min_trait)
        new_line.last_trait = min_trait
        created_lines.append(new_line)
    unused_traits = list(set(all_inds_from_chunk2) - set(used_traits_from_chunk2))

    for trait in unused_traits:
        chunk = chunk_2[trait]
        new_line = Line()
        start_trait = Trait(chunk_1[0].x_1, chunk.x_1, chunk.y_1)
        new_line.continue_line(start_trait)
        new_line.continue_line(chunk)
        new_line.last_trait = chunk
        created_lines.append(new_line)
    return created_lines

def connect_start_lines_with_next_chunks(lines, chunk, avg_height):
    all_inds_from_chunk = [i for i in range(len(chunk))]
    used_traits_from_chunk = []
    for line in lines:
        trait_1 = line.last_trait
        min_trait = Trait(trait_1.x_1, trait_1.x_2, 100)
        minimum = sys.maxsize
        for j, trait_2 in enumerate(chunk):
            if trait_1.dist(trait_2) <  minimum:
                minimum = trait_1.dist(trait_2)
                min_trait = trait_2
            if minimum >= avg_height / 2.0:
                min_trait = Trait(min_trait.x_1, min_trait.x_2, trait_1.y_1)
            if (min_trait.x_1 == trait_2.x_1) and (min_trait.x_2 == trait_2.x_2) \
                and (min_trait.y_1 == trait_2.y_1):
                used_traits_from_chunk.append(j)
        line.continue_line(min_trait)
        line.last_trait = min_trait
    unused_traits = list(set(all_inds_from_chunk) - set(used_traits_from_chunk))
    print(unused_traits)
    for i in unused_traits:
        trait = chunk[i]
        start_trait = Trait(0, trait.x_1, trait.y_1)
        new_line = Line()
        new_line.continue_line(start_trait)
        new_line.continue_line(trait)
        new_line.last_trait = trait
        lines.append(new_line)
        
def connect_lines_with_chunk(lines, chunk, avg_height):
    """ Combine the remaining chunk and already created lines"""
    for line in lines: 
        trait_1 = line.last_trait
        minimum = sys.maxsize
        if len(chunk) == 0:
            return
        for trait_2 in chunk:
            if trait_1.dist(trait_2) <  minimum:
                minimum = trait_1.dist(trait_2)
                min_trait = trait_2
            if minimum >=  avg_height / 1:
            
                min_trait = Trait(min_trait.x_1, min_trait.x_2, trait_1.y_1)
        line.continue_line(min_trait)
        line.last_trait = min_trait

#### Cut and draw lines 

In [72]:
def cut_line(image, line_1, line_2):
    n, m = image.shape
    img = image.copy()
    x_list = []
    for i, point in enumerate(line_1.data[:-1]):
        next_point = line_1.data[i + 1]
        img[:point.y, point.x:next_point.x] = 255
        x_list.append(point.x)
    for i, point in enumerate(line_2.data[:-1]):
        next_point = line_2.data[i + 1]
        img[point.y:n, point.x:next_point.x] = 255
        x_list.append(point.x)
    max_x = max(x_list)
    img[:, max_x:m] = 255
    inds = np.argwhere(img == 0)
    if len(inds) != 0:
        x_min = min(inds[:, 0])
        x_max = max(inds[:, 0])
        y_min = min(inds[:, 1])
        y_max = max(inds[:, 1])
        show_gray(img[x_min:x_max,:])
        return img[x_min:x_max, y_min:y_max]       

def cut_lines(image, created_lines):
    lines = []
    for i, line in enumerate(created_lines[:-1]):
        lines.append(cut_line(image, line, created_lines[i + 1]))
    return lines

def draw_line(image, line):
    img = image.copy()
    color = [random.randint(0, 255) for _ in range(3)]
    for i, point in enumerate(line.data[:-1]):
        next_point = line.data[i + 1]
        cv2.line(img, (point.x, point.y), (next_point.x, next_point.y), color ,3)
    return img
    
def draw_lines(image, lines, channels = 2):
    """Draw splitting lines"""
    img = image.copy()
    for line in lines:
        img = draw_line(img, line)
    if channels == 3:
        cv2.imwrite("12345.png", img)
        show(img)
    else:
        show_gray(img)

#### Words extraction from line

In [73]:
def get_words_from_line(line, min_width = 10, thresh =  100): #400000
    """
    line : grayscale line
    min_width : min space length
    """
    words = []
    if line is None:
        return words
    n, m = line.shape
    image = cv2.bitwise_not(line)
    y = np.sum(image // 255, axis = 0)

    _, inds = np.where([y == 0])

    start = 0
    spaces = []
    for i, ind in enumerate(inds[:-1]):
        if (ind + 1 == inds[i + 1]) and (start == 0):
            start = ind
        elif (ind + 1 < inds[i + 1]) and (start != 0):
            if (ind - start) >= min_width:
                spaces.append([int(start), int(ind)])
                start = 0
            else:
                start = 0

    spaces = np.ravel(spaces)
    spaces = np.insert(spaces, [0, len(spaces)], [0, m])
    spaces = spaces.reshape(len(spaces) // 2, 2)     
    
    for inds in spaces:
        word = line[:, int(inds[0]): int(inds[1])]
        print("sum", np.sum(word))
        #show_gray(word)
        if np.sum(1 - (word / 255)) > thresh:
            words.append(word)
    return words

def extract_words(lines):
    words = []
    for line in lines:
        words += get_words_from_line(line)
    return words

#### Resulted function to prepare an image

In [74]:
def contours_extraction(img_path, thresh_index):
    img = cv2.imread(img_path)
    h_img, w_img, _ = img.shape
    image = img[40:h_img-40, 40:w_img-40]
    #show(image)
    words = []
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    ret,thresh = cv2.threshold(gray,thresh_index,255,cv2.THRESH_BINARY_INV)
    #show(thresh)
    kernel = np.ones((5,5), np.uint8)
    img_dilation = cv2.dilate(thresh, kernel, iterations=1)
    ctrs, hier = cv2.findContours(img_dilation.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE )
    #sort contours
    sorted_ctrs = sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])
    resulted_rectangles = get_rectangles_from_contours(sorted_ctrs, h_img, w_img)
    
    # Show how resulted contours will be selected at image
    for rect in resulted_rectangles:
        cv2.rectangle(thresh,(rect.x,rect.y),( rect.x + rect.w, rect.y + rect.h ),(255,255,255),2)
    show(thresh)

    img = cv2.imread(img_path)
    image_to_cut = img[40:h_img-40, 40:w_img-40]
    
    #Add words into array
    for rect in resulted_rectangles:
        roi = image_to_cut.copy()[rect.y:rect.y+rect.h, rect.x:rect.x+rect.w]
        words.append(roi)
        
    return words
    
    #words.sort(key = sort_w, reverse = True)
    #res_words = []
    #for i, word in enumerate(words):
    #       res_words.append(word)
    #return res_words

In [75]:
def resulted_function(img_path, path_to_save):
    img = cv2.imread(img_path)
    index1 = img_path.index('/')+1
    index2 = img_path.index('.')
    author = img_path[index1:index2]
    h_img, w_img, _ = img.shape
    # Get optimal binarization index
    res_index = compare_thresh_indexes(img)
    print('res_index ' + str(res_index))
    
    contours = contours_extraction(img_path, res_index)
    for i, contour in enumerate(contours):
        h, w, _ = contour.shape
        binary_contour = get_thresh_image(contour, res_index)
        percent = percent_of_white_pixels_word(binary_contour)
        
        if 0.7 <= percent <= 0.95 :
            print('contour № ' + str(i))
            show(binary_contour)
            try:
                if h >=  w/5 or w <= w_img / 15:
                    words = prepare_binary_contour(binary_contour)
                    #if lines is not None:
                        #words = extract_words(lines)
                    for k, word in enumerate(words):
                        try:
                            h_lw, w_lw = word.shape
                            if h_lw >= h/5 and w_lw >= w/5:
                                path = path_to_save + '/' + author + '_' + 'word_' + str(i) + str(k) + '.png'
                                cv2.imwrite(path, word) 
                        except:
                            path = path_to_save + '/' + author + '_' + 'word_' + str(i)+ '.png'
                            cv2.imwrite(path,binary_contour) 
                else:
                    path = path_to_save + '/' + author + '_' + 'word_'+ str(i) + '.png'
                    cv2.imwrite(path,binary_contour)   
            except (ZeroDivisionError, statistics.StatisticsError):
                path = path_to_save + '/' + author + '_' + 'word_'+ str(i) + '.png'
                cv2.imwrite(path,binary_contour)             

            #path = path_to_save + '/' + author + '_' + 'word_' + str(i)+ '.png'
           # cv2.imwrite(path,binary_contour) 
    print(len(contours))

In [ ]:
path_for_image = ''
path_for_save = ''
resulted_function(path, path_for_save)